# Chapter 3 -- Types and Typeclasses


In [1]:
-- IHaskell settings
:opt no-lint
:opt no-pager

## Haskell type system:
- the type of _every_ expression is known at compile time
- has type inference: cleaner type annotations
- Types are always wrote with capital case
- To examine types, use `:t` command

In [2]:
:t 'a'
:t True
:t "Hello types !" -- `[]` denotes a list
:t (True, 'a')
:t 4 == 5

'a' :: Char

True :: Bool

"Hello types !" :: [Char]

(True, 'a') :: (Bool, Char)

4 == 5 :: Bool

Functions have types:

An Haskell programmer can choose to give a function an explicit type declation -- generally this is considered to be good practice except when writing very short functions

In [3]:
removeNonUppercase :: [Char] -> [Char] -- IHaskell doesn't need `let` for functions to have type declation
removeNonUppercase st = [ c | c <- st, c `elem` ['A'..'Z']]

Haskell can infer function type without type declation

In [4]:
removeNonUppercase' st = [ c | c <- st, c `elem` ['A'..'Z']]
:t removeNonUppercase'

removeNonUppercase' :: [Char] -> [Char]

Type declation for function with several parameters:
- there is no special distinction between the parameters and the return type

In [5]:
addThree :: Int -> Int -> Int -> Int
addThree x y z = x + y + z
addThree 1 2 3

6

## Common types

`Int`:
- on 32-bit machine: `-2^31+1` ~ `2^31-1`
- on 64-bit machine: `-2^63+1` ~ `2^63-1`
`Integer`: not bounded but less efficient

In [6]:
show (maxBound :: Int)
2^63 - 1
-- show (maxBound :: Integer) -- Will result in error !

"9223372036854775807"

9223372036854775807

In [7]:
factorial :: Int -> Int
factorial n = product [1..n]
factorial 50 -- `Int` overflows !

-3258495067890909184

In [8]:
factorial' :: Integer -> Integer
factorial' n = product [1..n]
factorial' 50 -- `Integer` doesn't overflows !

30414093201713378043612608166064768844377641568960512000000000000

- `Float`: a real floating point with single precision
- `Double`: a real floating point with double the precision

In [9]:
circumference :: Float -> Float
circumference r = 2 * pi * r
circumference 4.0

25.132742

In [10]:
circumference' :: Double -> Double
circumference' r = 2 * pi * r
circumference' 4.0

25.132741228718345

`Bool`: has only two values -- `True` and `False`

- `Char`: denoted by single quotes
- `[Char]`(`String`): denoted by double quotes

In [11]:
:t 'a'
:t "a"

'a' :: Char

"a" :: [Char]

Tuple types:
- varies by their length and the type of their components: there are theoretically infinite number of tuple types
- empty tuple has a type -- and the type only has a single value `()`

In [12]:
:t (1, '1')
:t ()

(1, '1') :: forall a. Num a => (a, Char)

() :: ()

## Type variables

In [13]:
:t head
:t fst

head :: forall a. [a] -> a

fst :: forall a b. (a, b) -> a

`a` and `b` are **type variable**s
- it's not written in capital case so it's not exactly a type.
- is much like generics in class-based OOP, but much more powerful than that
- lets us to write very general functions if they don't use any specific behaviour of the types in them
  * functions that have type variables are called ***polymorphic functions***

## Typeclass
- is a sort of _interface_ that defines some behaviour
  * if a type s a part of a typeclass, that means that it supports and implements the behavior the typeclass describes

In [14]:
:t (==)
-- NOTE:
-- for infix functions, we need to surround it in parentheses in order to
-- examine its type, pass it to another function or call it as a prefix function
-- e.g. `(==) 1 1`

(==) :: forall a. Eq a => a -> a -> Bool

`=>` symbol:
- everything before `=>` is called **class constraint**
- means `a` must be a member of the `Eq` class

`Eq` typeclass provides and interface for testing for equality
- Any type where it makes sense to test for equality between two values of that type should be a member of the `Eq` class

### Basic typeclasses

**`Eq`**:
- is used for types that support equality testing
- its members need to implement `==` and `/=`
  * if there is an `Eq` typeclass constraint for a variable in a function, then it uses `==` or `/=` somewhere inside its definition
- All standard Haskell types except for IO (the type for dealing with input and output) and functions are a part of the `Eq` typeclass

**`Ord`**:
- is for types that have an ordering
- covers all the standard comparing functions such as `>`, `<`, `>=` and `<=`
- the `compare` function takes two `Ord` members of the same type and returns an ordering:
- `Ordering` is a type that represent the "ordering":
  * it can be `GT`, `LT` or `EQ`, meaning _greater than_, _lesser than_ and _equal_, respectively

In [15]:
:t (>)
"Abrakadabra" `compare` "Zebra"
5 `compare` 3

(>) :: forall a. Ord a => a -> a -> Bool

LT

GT

**`Show`**:
- its member can be presented as a string
- the most used function that deals with the `Show` typeclass is `show` -- it takes a value whose type is a member of `Show` and presents it to us as a string

In [16]:
show 3
:t show True

"3"

show True :: String

**`Read`**:
- opposite typeclass of `Show` -- the `read` function takes a srring and returns a type which is a member of `Read`

In [17]:
read "True" || False
read "8.2" + 3.8
read (show 5) - 2
read "[1,2,3,4]" ++ [3]

True

12.0

3

[1,2,3,4,3]

NOTE:
- all the usages of `read` so far is where their result is used afterwards -- thus Haskell can infer the return type of `read`
- but just `read "1"` won't work, since Haskell doesn't know which type in `Read` typeclass we want.

In [18]:
read "1" -- Error !

: 

In [19]:
:t read -- the return type is a type variable !

read :: forall a. Read a => String -> a

We can use explicit **type annotations** for saying what the type of an expression should be. For type annotations, we use `::`

In [20]:
read "1" :: Int -- No error !
read "1" :: Float
:t (read "1" :: Float) * 4
read "(1, '1')" :: (Int, Char)

1

1.0

(read "1" :: Float) * 4 :: Float

(1,'1')

Where to use type annotation:
> Most expressions are such that the compiler can infer what their type is by itself.
> But sometimes, the compiler doesn't know whether to return a value of type `Int` or `Float` for an expression like `read "5"`.
> To see what the type is, Haskell would have to actually evaluate `read "5"`.
> **But since Haskell is a statically typed language, it has to know all the types before the code is compiled** (or in the case of GHCI, evaluated).
> So we have to tell Haskell: "Hey, this expression should have this type, in case you don't know!".

**`Enum`**:
- its members are sequentially ordered types -- they can be _enumerated_
- use cases:
  * list ranges
  * successor and predecesor: `succ` and `pred`
- types in this classes: `()`, `Bool`, `Char`, `Ordering`, `Int`, `Integer`, `Float`, `Double`

In [21]:
[LT .. GT]
succ 'B'
pred True
-- pred False -- will result in error

[LT,EQ,GT]

'C'

False

**`Bounded`** members have an upper and a lower bound

In [22]:
-- XXX: in IHaskell, the result of `maxBound :: T` is needed to be used in some way
show (minBound :: Int)
show (maxBound :: Char)

"-9223372036854775808"

"'\\1114111'"

NOTE: all tuples are part of `Bounded` if the components are also in it.

In [23]:
show (maxBound :: (Int, Char, Bool))

"(9223372036854775807,'\\1114111',True)"

**`Num`** is a numeric typeclass -- its members have the property of being able to act like numbers.

To join `Num`, a type must already be friends with `Show` and `Eq`

Actually whole numbers are polymorphic constants -- they can act like any type that's a member of the `Num` typeclass.

In [24]:
:t 20
20 :: Int
20 :: Integer
20 :: Float
20 :: Double

20 :: forall p. Num p => p

20

20

20.0

20.0

In [25]:
:t (*)

(*) :: forall a. Num a => a -> a -> a

`*` takes two numbers of **the same type** and returns a number of that type. So the behavior below.

In [26]:
(5 :: Int) * (6 :: Integer) -- Error

: 

In [27]:
5 * (6 :: Integer) -- No error !

30

- `Num` includes all numbers, including real and integral
- `Integral` only includes integral (whole) numbers
  * `Int` and `Integer` is in this typeclass
- `Floating` only includes floating point numbers
  * `Float` and `Double` is in this typeclass

**`fromIntegral`**:
- `fromIntegral :: forall a b. (Integral a, Num b) => a -> b`
- useful when we want integral and floating point types to work together nicely

E.g. add a length of a list to a floating point

In [28]:
(length [1, 2, 3]) + 3.2

: 

In [29]:
fromIntegral (length [1, 2, 3]) + 3.2

6.2

In [30]:
:t length

length :: forall (t :: * -> *) a. Foldable t => t a -> Int

NOTE:
- If `length` has a more general type `(Num b) => length :: [a] -> b`, then we don't need `fromIntegral` from the beginning
- but maybe there are historical reasons, huh